# 多语言机器学习
### 使用PatentData探索BLEU分数
#### Lee Mackey的

该笔记本随附[多语言机器学习](https://medium.com/@glmack)

## 介绍
---

您的机器学习中文吗？我不会说普通话或广东话，因此Google Translate可以将前面的句子翻译成“您和您的机器学习中文吗？但是研究人员如何快速评估机器翻译的质量？这个问题概括了引起BLEU指标的基本挑战。 BLEU代表双语语言学习，它是机器翻译质量的默认量度，有时还用于[跨语言](*https://arxiv.org/abs/1901.07291) 自然语言处理（NLP）任务。该度量标准在机器翻译领域已经确立，但是一些分析家质疑该算法是否适用于该度量标准最初目标之外的更多任务。本文使用从多语言专利文档中摘录的示例，初步探讨了BLEU的课程，实现和局限性.

Does your machine learn in Chinese? I don't speak Mandarin or Cantonese so Google Translate gets all the credit \- good or bad \- for translating the preceding sentence into: "您和您的機器學習中文嗎?" But how might a researcher quickly evaluate the quality of machine translations? This question encapsulates the basic challenge that gives rise to the BLEU metric. BLEU, which stands for bilingual language understudy, is a default measure of machine translation quality and is also sometimes applied to [cross-lingual](https://arxiv.org/abs/1901.07291) natural language processing (NLP) tasks. The metric is well-established in the machine translation space but some analysts question the algorithm's applicability to a wider set of tasks beyond the measure's original objectives. This article takes an initial dive into the lessons, implementations and limits of BLEU using examples drawn from multilingual patent documents.

In [1]:
# 导入模块
from multilingual_machines import (split_tokens, 
                                   clean_punctuation,
                                   represent_references)
from nltk.translate.bleu_score import corpus_bleu
from nltk import bigrams, trigrams, ngrams
import string
import json
from IPython.display import IFrame

In [2]:
# 从Github存储库中的'patent_examples.txt'文件中加载示例数据
with open('patent_examples.txt') as f:
    data = json.load(f)

## BLEU基础
---

IBM的研究人员于2002年开发了BLEU算法，作为一种参考基准人工翻译来评估机器翻译质量的有效方法。如果您对...的创始背景和目标感兴趣，那么开发人员Papineni和同事的原始[纸](https://www.aclweb.org/anthology/P02-1040.pdf) 是一个不错的起点。算法。

In [3]:
# 浏览原始BLEU纸
IFrame('https://www.aclweb.org/anthology/P02-1040.pdf', width=650, height=300)

BLEU是对“候选”机器翻译与一个或多个“参考”人工翻译之间匹配的单词序列的调整精度度量。 BLEU在与人工翻译中的n-gram相匹配的机器翻译中对“ n-grams”（长度为*n*的单词序列的术语）进行计数，除以机器中n-grams的总数翻译。调整措施的方式是，将匹配项裁剪到人类翻译中n-gram出现的最大次数，并且还惩罚了单词长度与参考翻译不同的机器翻译。

最终的BLEU分数是介于0和1之间的数字，其中0表示候选文本和参考文本之间的零个n-gram匹配，并且1等于等于参考文献之一的机器翻译。实际上，量度计数在多个单词序列长度上匹配-4克（四个单词的序列），三元语法（三个单词的序列），二元语法（两个单词的序列）和单字母语法（一个-单词序列）--通过各个n-gram计算的几何平均值。该算法设计用于在句子语料库级别进行比较，并在句子的基本单元上计算n-gram匹配，然后将其组合为语料库级别得分。为了澄清术语，本文中术语“文档”的使用是指句子的语料库。如果您对了解该算法的其他资源感兴趣，可以在deeplearning.ai上查阅[影片教学](https://www.youtube.com/watch?v=DejHQYAGb7Q)，其中讨论了该算法的详细信息或machinelearningmastery.com上的[教程](https://machinelearningmastery.com/calculate-bleu-score-for-text-python/) 探索NLTK实现。为了切实探索度量标准的用例，我接下来使用中文专利翻译来应用BLEU。

##将BLEU应用于专利文本
---

根据WIPO（世界）最近的一份[报告](https://www.wipo.int/publications/en/details.jsp?id=4386)，用中文书写和提交的专利在机器学习领域的份额越来越大。知识产权组织），这是管理专利的全球组织。为了探索BLEU在这种多语言空间中的基础知识，您首先可以从中国的NLP创新国内专利开始，该专利已由电子商务公司阿里巴巴扩展到了全球。中文专利的标题显示在下面。

#### Acquire translations

In [4]:
# 检查中文示例专利的标题
print(data['original_title_cn'])

['机器处理及文本纠错方法和装置、计算设备以及存储介质']


有关示例专利的更多详细信息，WIPO的数据查询工具同时提供[英语](https://patentscope.wipo.int/search/en/detail.jsf?docId=WO2019085779) 和[中文](https://patentscope您可以在浏览器中检查的.wipo.int/search/zh/detail.jsfdocId=WO2019085779) 语言版本。

In [5]:
# 使用WIPO GUI检查专利的中文版本
IFrame('https://patentscope.wipo.int/search/zh/detail.jsf?docId=WO2019085779',
       width=650,
       height=300)

# 交换URL或粘贴到浏览器中以使用WIPO GUI检查英语版本
# https://patentscope.wipo.int/search/en/detail.jsf?docId=WO2019085779

人类专家通常会提供同等质量的翻译，但其词选择和结构却有所不同。因此，BLEU接受单个或多个人工翻译作为比较参考。接下来，我通过[Gengo](https://gengo.com/)从两个不同的人类翻译人员那里获得了两个中文译成的中文专利的中文专利翻译，这是一个基于Web的人类翻译平台。

In [6]:
# 检查人员＃1专利摘要的从Ch到En的翻译
reference_human1_summary = data['reference_human1_summary']
print(reference_human1_summary)

['The invention discloses a machine processing and text error correction method and device, a computing device, and a storage medium, specifically comprising corrected and rewritten text pairs of incorrect text and corresponding correct text.', 'The corrected and rewritten text pairs serving as a training corpus to train the machine processing model, thereby preparing a machine processing model suitable for text error correction.', 'Through extraction of corrected and rewritten text pairs from a log, the machine processing model can be trained and thus made fit for text correction by inputting the first text into the machine processing model to get the second text, that is the error correction result text.', 'In addition, the language model or the common lexicon can be used to determine whether the first text needs to be corrected.', 'The training corpus extracted from a log can be used to train the language model, or the common lexicon can be sorted by segmenting and counting text in 

In [7]:
# 检查人类＃2的专利翻译
reference_human2_summary = data['reference_human2_summary']
print(reference_human2_summary)

['This invention makes public a machine processing and text error correction method and hardware, computing equipment and storage medium, and specifically pairs error text with the corresponding corrected and modified correct text.', 'It uses this text pair as training material for the machine processing model, and from there prepares the machine processing model that is applied to the text correction.', 'It can train the machine processing model using a diary or daily journal and make it suitable for text correction.', 'The first text version is inputted into the machine processing model to get the second text version, which is the corrected text.', 'Additionally, it can also use a stored language model or common vocabulary bank to determine if the first text version needs correction.', 'It can use the practice language material gathered from the diary or daily journal to train the language model, and it can also initialize the common vocabulary bank through the segmentation and analy

接下来，您可以从以下两种单独的机器学习算法中获取“候选”机器翻译：Google翻译和世界知识产权组织（WIPO），如下所示。

In [8]:
# 检查Google Translate的Ch-to-En机器翻译专利
candidate_google_summary = data['candidate_google_summary']
print(candidate_google_summary)

['The invention discloses a machine processing and text error correction method and device, a computing device and a storage medium, and particularly comprises an error correction rewriting pair of an error text and a corresponding correct text, and an error correction rewriting pair as a training corpus, and a machine processing model.', 'Training is performed, thereby preparing a machine processing model suitable for text correction. The machine processing model can be trained to mine the error correction by mining the error correction rewrite pair from the log.', 'The first text is input into the machine processing model to obtain a second text, that is, an error correction result text.', 'In addition, you can use the language model or common lexicon to determine whether the first text needs to be corrected.', ' The language model can be trained using the training corpus extracted from the log, or the common lexicon can be organized by segmenting and counting the text in the log.', 

In [9]:
# 检查WIPO对WIPO的Ch-to-En机器翻译
candidate_wipo_summary = data['candidate_wipo_summary']
print(candidate_wipo_summary)

['The present invention discloses a machine processing and text correction method and device, computing equipment and a storage medium.', 'Specifically comprising corrected and rewritten text pairs of incorrect text and corresponding correct text, the corrected and rewritten text pairs serving as a training corpus for training a machine processing model, and in this way developing a machine processing model for use in text correction.', 'Through extraction of corrected and rewritten text pairs from a log, the machine processing model can be trained and thus made fit for text correction by inputting a first text into the machine processing model to obtain a second text i.e. a corrected text result.', 'Moreover, a language model or a lexicon of commonly used words can be used to assess whether text needs correction. The training corpus extracted from the log can be used to train the language model and also, through text segmentation and statistical analysis of text in the log compile a l

#### 计算BLEU分数

BLEU有多种实现和扩展，例如流行的[sacreBLEU](https://github.com/mjpost/sacreBLEU)软件包。本示例首先使用[bleu score 模组](https://www.nltk.org/api/nltk.translate.html?highlight=bleu#module-nltk.translate.bleu_score)（在自然语言工具包（NLTK）中，该工具包基于Papineni等人的原始实现。 “ corpus_bleu”函数接受参考和候选翻译，它们表示为单个单词标记，您可以通过标准的NLP预处理步骤来创建它们。

In [10]:
# 将候选翻译和参考翻译拆分为标记
ref_human1_summary = split_tokens(reference_human1_summary)
ref_human2_summary = split_tokens(reference_human2_summary)
can_google_summary = split_tokens(candidate_google_summary)
can_wipo_summary = split_tokens(candidate_wipo_summary)

In [11]:
# 从候选和引用中的令牌中清除标点符号
ref_human1_summary = clean_punctuation(ref_human1_summary)
ref_human2_summary = clean_punctuation(ref_human2_summary)
can_google_summary = clean_punctuation(can_google_summary)
can_wipo_summary = clean_punctuation(can_wipo_summary)

对于具有多个句子的文档，原始的BLEU实现逐个句子计算n-gram匹配项，然后将所有候选句子的裁剪后的n-gram计数求和，最后除以文档中候选n-gram的数量。正如NLTK文档所指出的那样：原始的BLEU度量标准“计算微观平均精度（即，除法前将每个假设-参考对的分子和分母相加）”。默认情况下，“ corpus_bleu”计算相等加权的n元语法分数的几何平均值，该分数包括4元，三元，二元和unigram。

In [12]:
# 符串＃1的翻译中检查二元语法示例
bi_grams = list(ngrams(ref_human1_summary[0], 2))[0:10]

In [13]:
# 从人类＃1的翻译字符串中检查4克示例
four_grams = list(ngrams(ref_human1_summary[0], 4))[0:10]

print(f"bi-gram tuples: {bi_grams}", '\n')
print(f"4-grams tuples: {four_grams}")

bi-gram tuples: [('The', 'invention'), ('invention', 'discloses'), ('discloses', 'a'), ('a', 'machine'), ('machine', 'processing'), ('processing', 'and'), ('and', 'text'), ('text', 'error'), ('error', 'correction'), ('correction', 'method')] 

4-grams tuples: [('The', 'invention', 'discloses', 'a'), ('invention', 'discloses', 'a', 'machine'), ('discloses', 'a', 'machine', 'processing'), ('a', 'machine', 'processing', 'and'), ('machine', 'processing', 'and', 'text'), ('processing', 'and', 'text', 'error'), ('and', 'text', 'error', 'correction'), ('text', 'error', 'correction', 'method'), ('error', 'correction', 'method', 'and'), ('correction', 'method', 'and', 'device')]


这两个机器翻译示例的BLEU分数计算结果是什么？

In [15]:
# 将Google和WIPO翻译中的每个句子与两个参考文献中的所有句子进行比较

# 表示对6篇句子的WIPO翻译的引用
refs_list_6 = represent_references(ref_human1_summary, ref_human2_summary, 6)

# 表示对5篇句子的WIPO翻译的引用
refs_list_5 = represent_references(ref_human1_summary, ref_human2_summary, 5)

结果表明，与Gengo上的人工翻译的参考翻译相比，Google和WIPO的翻译质量相似。如下所示，由Google Translate进行的首次候选翻译的得分为0.53，而WIPO的得分为0.54。

In [ ]:
# 计算Google翻译的BLEU分数
bleu_google = round(corpus_bleu(refs_list_6, can_google_summary[:]), 2)

# 计算WIPO翻译的BLEU分数
bleu_wipo = round(corpus_bleu(refs_list_5, can_wipo_summary[:]), 2)

print(f"Google Translate BLEU score: {bleu_google}")
print(f"WIPO BLEU score: {bleu_wipo}")

考虑到BLEU应用的这一实际示例，有助于确定算法对机器翻译和自然语言处理任务适用性的潜在范围。

## BLEU的限制
---

研究人员普遍认为，BLEU可方便地评估机器翻译系统。但是，一些研究人员警告说，BLEU可能不适用于机器翻译的某些方面或更广泛的NLP任务。一个关键的批评是，BLEU分数可能不一定与人工翻译中的明显质量差异有很好的相关性。正如Callison-Burch及其同事在一篇引人注目的[论文](https://www.aclweb.org/anthology/E06-1032/) 中指出的那样：“在某些情况下，BLEU的改善不足以反映真实的改进翻译质量，而在其他情况下，为了显着提高翻译质量，不必改进BLEU。”这些作者还建议，BLEU可能不适合比较使用不同技术的机器翻译系统，检测BLEU无法建模的翻译方面或评估测试语料库中的不常见改进。

对BLEU进行文献综述的研究人员在最近的一篇论文中认为 [纸](https://www.mitpressjournals.org/doi/full/10.1162/COLI_a_00322) ，BLEU与人的相关性可能随评估系统的详细信息，使用的特定语料库文本以及用于人工评估的确切协议。作者，Reiter及其同事建议，应该通过现实世界的人类评估（例如A / B测试）来评估BLEU，以确定BLEU与现实有效性可靠相关的环境。基于文献综述的结果，这些作者告诫研究人员应该将BLEU用作系统级别机器翻译的诊断程序，而不是作为评估系统输出的评估技术。文献表明人们普遍认识到该措施的实用性和不足之处，一些研究人员，例如最近的论文 [纸](https://arxiv.org/abs/1804.08771) 的作者，将重点放在了提高措施的清晰度上。报告BLEU分数。例如，这将包括让研究人员包括预处理方案和其他标准化惯例，以报告BLEU评分的实施情况和评分细节。这些论文中提出的观点在使用和解释专利领域以及不同用例中的BLEU分数时应谨慎注意。

##结论
---
本文概述了BLEU分数的目标，示例和限制。要在Jupyter笔记本中运行本文附带的完整Python代码，您可以转到Github [资料库](https://github.com/glmack/multilingual_machines) 进行多语言机器学习。有关我对这个领域的兴趣的更多背景，请访问我的网站[lee-mackey.com](http://lee-mackey.com)。随着机器学习领域的专利文档（尤其是中文专利）越来越多的语言使用，本文提出的观点有望帮助您将这种算法应用于您的特定用例时浮出水面并根据上下文进行考虑。如果您在NLP工作流程中使用多种语言，则了解BLEU的这些详细信息将有助于选择是否，何时以及如何在项目中使用此指标。